# Лабораторная работа  "Подсчет кэш-промахов"

### 1. Определим параметры кэшей CPU c помощью утилиты cpuinfo_x86:

```
...
# Caches
## L1 Instruction Cache
Size                  : 32K
Line Size             : 64B
Sharing               : shared between 2 processor threads
Sets                  : 64
Partitions            : 1
Associativity         : 8

## L1 Data Cache
Size                  : 32K
Line Size             : 64B
Sharing               : shared between 2 processor threads
Sets                  : 64
Partitions            : 1
Associativity         : 8

## L2 Unified Cache
Size                  : 256K
Line Size             : 64B
Sharing               : shared between 2 processor threads
Sets                  : 512
Partitions            : 1
Associativity         : 8

## L3 Unified Cache
Size                  : 3M
Line Size             : 64B
Sharing               : shared between 16 processor threads
Sets                  : 4096
Partitions            : 1
Associativity         : 12
...
```

### 2. Определим параметры десктопа.
Параметры CPU:

``` 
# Identification
Vendor                : GenuineIntel
Brand String          : Intel(R) Core(TM) i5-4258U CPU @ 2.40GHz
Model Number          : 69
Family Code           : 6
Extended Model        : 4
Extended Family       : 0
Stepping ID           : 1
Signature             : 263761

# Address Bits
Physical Addressing   : 39
Virtual Addressing    : 48

# Multi-Core Information
Logical Processors (Threads) per Physical Processor : 16
Cores per Physical Package                          : 8
```


Общие параметры:

![title](system_info_mac.png)

### 3. Код анализа количества кэш промахов

[main.cpp](file/main.cpp)

```cpp
class Cache {
    public:
        long long hitCounter = 0;
        long long missCounter = 0;

        Cache(int cacheLineSize, int channels, int size) : cacheLineSize(cacheLineSize), channels(channels),
                                                           size(size) {
            totalLinesInCache = size / cacheLineSize;
            linesInChannel = totalLinesInCache / channels;

            cout << "cache size " << size << " B" << endl;
            cout << "channels " << channels << endl;
            cout << "cacheLineSize " << cacheLineSize << " B" << endl;
            cout << "totalLinesInCache " << totalLinesInCache << endl;
            cout << "linesInChannel " << linesInChannel << endl;

            cache = new long long[totalLinesInCache];
            lastAccessTime = new long long[totalLinesInCache];
            for (int i = 0; i < totalLinesInCache; i++) {
                cache[i] = -1;
                lastAccessTime[i] = -1;
            }
        }

        virtual ~Cache() {
            delete[] cache;
            delete[] lastAccessTime;

        }

        void access(long long addrInMem) {
            long long lineNumberInMem = getCacheLineNumber(addrInMem);
            int lineTag = (int) (lineNumberInMem % linesInChannel);

            if (isCacheContain(lineTag, lineNumberInMem)) {
                hitCounter++;
            } else {
                missCounter++;
                addLineToCache(lineTag, lineNumberInMem);
            }
            return;
        }

    private:
        int cacheLineSize;
        int channels;
        int size;

        int linesInChannel;
        int totalLinesInCache;

        long long *cache;
        long long *lastAccessTime;

        long long getCurrentTimestamp() {
            return chrono::system_clock::now().time_since_epoch().count();
        }

        bool isCacheContain(int cacheLineTag, long long cacheLineNumberInMem) {
            for (int i = 0; i < channels; i++) {
                long n = cacheLineTag * channels + i;
                if (cache[n] == cacheLineNumberInMem) {
                    lastAccessTime[n] = getCurrentTimestamp();
                    return true;
                }
            }
            return false;
        }

        void addLineToCache(int cacheLineTag, long long cacheLineNumberInMem) {
            long lruLineNum = cacheLineTag * channels;
            long minAccessTime = lastAccessTime[lruLineNum];
            for (int i = 1; i < channels; i++) {
                long n = cacheLineTag * channels + i;
                if (lastAccessTime[n] < minAccessTime) {
                    minAccessTime = lastAccessTime[n];
                    lruLineNum = n;
                }
            }

            cache[lruLineNum] = cacheLineNumberInMem;
            lastAccessTime[lruLineNum] = getCurrentTimestamp();
        }

        long long getCacheLineNumber(long long addrInMem) {
            return addrInMem / cacheLineSize;
        }
    };

```

### 4. Графики сравнения количества кэш-промахов, предсказанных valgrind и собственным анализатором